###Linear Regression Example
Let's walk through the steps of the official documentation example. 
Doing this will help your ability to read from the documentation, understand it, and then apply it to your own problems (the upcoming Consulting Project).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Lenear Regression").getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [6]:
training = spark.read.format("libsvm").load("G:\Downloads Ex\Python-and-Spark-for-Big-Data-master\Python-and-Spark-for-Big-Data-master\Spark_for_Machine_Learning\Linear_Regression\sample_linear_regression_data.txt")

Interesting! We haven't seen libsvm formats before. In fact the aren't very popular when working with datasets in Python, but the Spark Documentation makes use of them a lot because of their formatting. Let's see what the training data looks like:

In [7]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

This is the format that Spark expects. Two columns with the names "label" and "features". 

The "label" column then needs to have the numerical label, either a regression numerical value, or a numerical value that matches to a classification grouping. Later on we will talk about unsupervised learning algorithms that by their nature do not use or require a label.

The feature column has inside of it a vector of all the features that belong to that row. Usually what we end up doing is combining the various feature columns we have into a single 'features' column using the data transformations we've learned about.

In [9]:
# These are the default values for the featuresCol, labelCol, predictionCol
lr = LinearRegression(featuresCol="features", labelCol = 'label', predictionCol='Predictions')

# You could also pass in additional parameters for regularization, do the reading 
# in ISLR to fully understand that, after that its just some simple parameter calls.
# Check the documentation with Shift+Tab for more info!

In [10]:
#fit the model 
lrModel = lr.fit(training)

In [14]:
lrModel.coefficients


DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [15]:
lrModel.intercept

0.14228558260358093

In [16]:
training_summary = lrModel.summary

In [17]:
training_summary.r2

0.027839179518600154

###Train/Test Splits
But wait! We've committed a big mistake, we never separated our data set into a training and test set. Instead we trained on ALL of the data, something we generally want to avoid doing. Read ISLR and check out the theory lecture for more info on this, but remember we won't get a fair evaluation of our model by judging how well it does again on the same data it was trained on!

Luckily Spark DataFrames have an almost too convinient method of splitting the data! Let's see it:

In [19]:
all_data = spark.read.format("libsvm").load("G:\Downloads Ex\Python-and-Spark-for-Big-Data-master\Python-and-Spark-for-Big-Data-master\Spark_for_Machine_Learning\Linear_Regression\sample_linear_regression_data.txt")

In [20]:
# Pass in the split between training/test as a list.
# No correct, but generally 70/30 or 60/40 splits are used. 
# Depending on how much data you have and how unbalanced it is.

train_data, test_data = all_data.randomSplit([0.7,0.3])

In [21]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                369|
|   mean| 0.2782555719861968|
| stddev| 10.243794333235705|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [22]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                132|
|   mean|0.19715904436957046|
| stddev| 10.561534492778808|
|    min|-22.837460416919342|
|    max| 26.903524792043335|
+-------+-------------------+



In [23]:
correct_model = lr.fit(train_data)

Now we can directly get a .summary object using the evaluate method:

In [24]:
test_result = correct_model.evaluate(test_data)

In [27]:
test_result.residuals.show()
test_result.rootMeanSquaredError

+-------------------+
|          residuals|
+-------------------+
|-21.507706705642306|
|-22.153022536156918|
|-21.008514612887016|
|-21.596530216692734|
| -18.92597929255892|
| -18.17341677691308|
|-19.467280060556806|
| -17.42914233788011|
|-17.262855045493527|
| -16.57816865061576|
|-16.754288046483243|
|-18.785745813269962|
|  -17.1833389112424|
| -14.68571457410269|
|-13.927374080276154|
|-15.312348364009804|
|-17.452744518775354|
| -13.95602513601062|
|-12.262970789451824|
|-11.698612414739186|
+-------------------+
only showing top 20 rows



10.528334817979239

In [28]:
unlabeled_data = test_data.select("features")

In [29]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [30]:
predictions = correct_model.transform(unlabeled_data)

In [32]:
predictions.show()

+--------------------+--------------------+
|            features|         Predictions|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -1.3297537112770343|
|(10,[0,1,2,3,4,5,...|  0.7206347719911121|
|(10,[0,1,2,3,4,5,...|  1.2257518232724787|
|(10,[0,1,2,3,4,5,...|  2.1941941864781835|
|(10,[0,1,2,3,4,5,...| 0.08005681966033878|
|(10,[0,1,2,3,4,5,...|  0.6792164200297371|
|(10,[0,1,2,3,4,5,...|  2.0386054896172996|
|(10,[0,1,2,3,4,5,...| 0.10242160520416016|
|(10,[0,1,2,3,4,5,...| 0.19745541961751195|
|(10,[0,1,2,3,4,5,...|-0.44832361359378986|
|(10,[0,1,2,3,4,5,...| 0.03519121287815469|
|(10,[0,1,2,3,4,5,...|   2.093538791958855|
|(10,[0,1,2,3,4,5,...|  0.9219086357896706|
|(10,[0,1,2,3,4,5,...|  -0.663156581276563|
|(10,[0,1,2,3,4,5,...| -1.3836065091401344|
|(10,[0,1,2,3,4,5,...|  0.5495901110786763|
|(10,[0,1,2,3,4,5,...|  3.1237660096999136|
|(10,[0,1,2,3,4,5,...| 0.08893724085185123|
|(10,[0,1,2,3,4,5,...| -0.8903648169137073|
|(10,[0,1,2,3,4,5,...| -1.074614